In [5]:
import pandas as pd
import os 
from pathlib import Path

In [9]:
os.getcwd()

'/Users/macintosh/github/data-science-project'

In [28]:
INPUT_DIR = "/Users/Macintosh/Downloads/predict-energy-behavior-of-prosumers/"
# INPUT_DIR = Path("/Users/Macintosh/Downloads/predict-energy-behavior-of-prosumers/")


In [29]:
train_df = pd.read_csv(INPUT_DIR + "train.csv", sep= ",")
# train_df = pd.read_csv(INPUT_DIR / "train.csv", sep= ",")

In [37]:
train_df.shape

(2018352, 9)

In [38]:
pd.value_counts(train_df.dtypes)

int64      7
float64    1
object     1
Name: count, dtype: int64